In [1]:
#imports 
import os
import h5py
import time
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline


In [2]:
#let's look at effect of flight profile factors in isolation
### Set-up - Define file location
filename = 'N-CMAPSS_DS02-006.h5'

In [3]:
# Time tracking, Operation time (min):  0.003
t = time.process_time()  

# Load data
with h5py.File(filename, 'r') as hdf:
        # Development set
        W_dev = np.array(hdf.get('W_dev'))             # W
        X_s_dev = np.array(hdf.get('X_s_dev'))         # X_s
        X_v_dev = np.array(hdf.get('X_v_dev'))         # X_v
        T_dev = np.array(hdf.get('T_dev'))             # T
        Y_dev = np.array(hdf.get('Y_dev'))             # RUL  
        A_dev = np.array(hdf.get('A_dev'))             # Auxiliary

        # Test set
        W_test = np.array(hdf.get('W_test'))           # W
        X_s_test = np.array(hdf.get('X_s_test'))       # X_s
        X_v_test = np.array(hdf.get('X_v_test'))       # X_v
        T_test = np.array(hdf.get('T_test'))           # T
        Y_test = np.array(hdf.get('Y_test'))           # RUL  
        A_test = np.array(hdf.get('A_test'))           # Auxiliary
        
        # Varnams
        W_var = np.array(hdf.get('W_var'))
        X_s_var = np.array(hdf.get('X_s_var'))  
        X_v_var = np.array(hdf.get('X_v_var')) 
        T_var = np.array(hdf.get('T_var'))
        A_var = np.array(hdf.get('A_var'))
        
        # from np.array to list dtype U4/U5
        W_var = list(np.array(W_var, dtype='U20'))
        X_s_var = list(np.array(X_s_var, dtype='U20'))  
        X_v_var = list(np.array(X_v_var, dtype='U20')) 
        T_var = list(np.array(T_var, dtype='U20'))
        A_var = list(np.array(A_var, dtype='U20'))
                          
W = np.concatenate((W_dev, W_test), axis=0)  
X_s = np.concatenate((X_s_dev, X_s_test), axis=0)
X_v = np.concatenate((X_v_dev, X_v_test), axis=0)
T = np.concatenate((T_dev, T_test), axis=0)
Y = np.concatenate((Y_dev, Y_test), axis=0) 
A = np.concatenate((A_dev, A_test), axis=0) 
    
print('')
print("Operation time (min): " , (time.process_time()-t)/60)
print('')
print ("W shape: " + str(W.shape))
print ("X_s shape: " + str(X_s.shape))
print ("X_v shape: " + str(X_v.shape))
print ("T shape: " + str(T.shape))
print ("A shape: " + str(A.shape))


Operation time (min):  0.03138361666666666

W shape: (6517190, 4)
X_s shape: (6517190, 14)
X_v shape: (6517190, 14)
T shape: (6517190, 10)
A shape: (6517190, 4)


In [25]:
X_t=np.concatenate((A[:,:2],T),axis=1)

In [5]:
X_w=np.concatenate((A[:,:2],W),axis=1)


In [6]:
#approach 3: sliding window across all flights and units, #kernel samples from each flight
def reshape(arr,kernel,window): #recall this was built with X_c in mind!!
    #data points are sliding windows size w
    #first grab number of flights
    f=0
    for i in np.unique(arr[:,0]):
        dub=arr[arr[:,0]==i]
        f+=np.max(dub[:,1])
    trim=arr[:,2:] #get rid of indices
    k=trim.shape[1] #only after you've deleted unnecessary features!!!;
    X=np.zeros((int(kernel*f),window,k))
    y=np.zeros((int(kernel*f),))
    t_ticker=0
    from sklearn.preprocessing import MinMaxScaler
    #indexer=[2,4,6,13,17,28]
    for i,n in enumerate(np.unique(arr[:,0])):
        dub=arr[arr[:,0]==n] #unit
        for j in np.unique(dub[:,1]):
            bub=dub[dub[:,1]==j] #flight
            sub=MinMaxScaler().fit_transform(bub)
            #sub=dub[dub[:,1]==j]
            #t_tocker=0
            t_tocker=int(len(sub)//2-kernel//2)
            for k in range(kernel):
                rub=sub[t_tocker:(t_tocker+window),:]
                X[t_ticker,:,:]+=rub[:,2:] #should be dim (window,k)
                y[t_ticker]+=len(sub)-(t_tocker+window)
                t_ticker+=1
                t_tocker+=1
    return X,y

In [8]:
X,y=reshape(X_w,kernel=100,window=50)

In [11]:
#imports for LSTM
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import MeanSquaredError,R2Score
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import load_model

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [11]:
X_tr=X
y_tr=y

In [12]:
a=np.arange(len(X_tr))
tr_ind=np.random.randint(0,len(X_tr),int(0.8*len(X_tr)))
te_ind=np.delete(a,tr_ind)
X_train=X_tr[tr_ind]
y_train=y_tr[tr_ind]
X_test=X_tr[te_ind]
y_test=y_tr[te_ind]

In [14]:
#construct RNN
model=Sequential() #each xi is run of n timestamps and we are targeting RUL for each 
model.add(LSTM(100,return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2]))) #input layer
model.add(LSTM(100,return_sequences=False)) #hidden layer
model.add(Dense(100,'relu')) #activation function
model.add(Dense(1,'linear')) #output
cp=ModelCheckpoint('modelnew.h5',save_best_only=True) #to save the best model
model.compile(loss='MeanSquaredError',optimizer=Adam(learning_rate=0.01),metrics=['R2Score']) #set the loss function optimizer and metric
model.fit(X_train,y_train,validation_split=0.3,epochs=25,callbacks=[cp]) #fit the model
model=load_model('modelnew.h5')

#X and y appear to be what I want them to be, scaled, matching.
#something about the setup of this model?
#worth a deep dive into the architecture


Epoch 1/25
1134/1134 [==============================] - 39s 34ms/step - loss: 5556395.0000 - r2_score: -0.2632 - val_loss: 4377799.5000 - val_r2_score: -4.9770e-04
Epoch 2/25
1134/1134 [==============================] - 39s 34ms/step - loss: 4195027.0000 - r2_score: 0.0463 - val_loss: 2442536.2500 - val_r2_score: 0.4418
Epoch 3/25
1134/1134 [==============================] - 39s 34ms/step - loss: 1949032.2500 - r2_score: 0.5569 - val_loss: 1426321.0000 - val_r2_score: 0.6740
Epoch 4/25
1134/1134 [==============================] - 37s 32ms/step - loss: 1567150.3750 - r2_score: 0.6437 - val_loss: 1298904.0000 - val_r2_score: 0.7031
Epoch 5/25
1134/1134 [==============================] - 37s 33ms/step - loss: 1238511.1250 - r2_score: 0.7184 - val_loss: 1120481.7500 - val_r2_score: 0.7439
Epoch 6/25
1134/1134 [==============================] - 38s 34ms/step - loss: 1123384.5000 - r2_score: 0.7446 - val_loss: 978444.6875 - val_r2_score: 0.7764
Epoch 7/25
1134/1134 [=========================

In [16]:
preds=model.predict(X_test)

908/908 [==============================] - 9s 10ms/step


In [15]:
#let's simulate two different routes KSEA-KJFK
#route 1 direct 36k
#route 2 around storm 38k, colder inlet temps, higher power settings, higher airspeed
#we can use our autoencoder to build these routes!

In [8]:
#autoencoder for early anomaly detection
#let's focus on "longer" flights >5000 timestamps
#add index columns to help in restructuring
X_c=np.concatenate((A[:,:2],X_s,X_v),axis=1)

In [9]:
#we'll try univariate first
from sklearn.preprocessing import MinMaxScaler
l=[]
window=1000
for i in np.unique(X_c[:,0]):
    dub=X_c[X_c[:,0]==i]
    for j in np.unique(dub[:,1]):
        sub=MinMaxScaler().fit_transform(dub[dub[:,1]==j])
        if len(sub)>3000 and len(sub)<5000:
            #ind=np.linspace(0,len(sub),5001).astype(int)[:-1]
            for k in range(2000):
                l.append(sub[k:k+window,5])
X=np.stack(l)
X=X.reshape(X.shape[0],X.shape[1],1)

In [13]:
from tensorflow.keras import layers
model = keras.Sequential(
    [
        layers.Input(shape=(X.shape[1], X.shape[2])),
        layers.Conv1D(
            filters=20,
            kernel_size=8,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1D(
            filters=20,
            kernel_size=8,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Conv1DTranspose(
            filters=20,
            kernel_size=8,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Dropout(rate=0.2),
        layers.Conv1DTranspose(
            filters=20,
            kernel_size=8,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Conv1DTranspose(filters=1, kernel_size=10, padding="same"),
    ]
)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss="mae")

In [14]:
history = model.fit(
    X,
    X,
    epochs=10,
    batch_size=300,
    validation_split=0.2,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, mode="min")
    ],
)

Epoch 1/10
64/64 [==============================] - 4s 62ms/step - loss: 0.2122 - val_loss: 0.1138
Epoch 2/10
64/64 [==============================] - 4s 62ms/step - loss: 0.0860 - val_loss: 0.0258
Epoch 3/10
64/64 [==============================] - 4s 60ms/step - loss: 0.0308 - val_loss: 0.0335
Epoch 4/10
64/64 [==============================] - 4s 60ms/step - loss: 0.0256 - val_loss: 0.0351
Epoch 5/10
64/64 [==============================] - 4s 60ms/step - loss: 0.0234 - val_loss: 0.0282
Epoch 6/10
64/64 [==============================] - 4s 60ms/step - loss: 0.0222 - val_loss: 0.0324
Epoch 7/10
64/64 [==============================] - 4s 62ms/step - loss: 0.0209 - val_loss: 0.0300
Epoch 8/10
64/64 [==============================] - 4s 67ms/step - loss: 0.0201 - val_loss: 0.0291
Epoch 9/10
64/64 [==============================] - 4s 63ms/step - loss: 0.0236 - val_loss: 0.0263
Epoch 10/10
64/64 [==============================] - 4s 62ms/step - loss: 0.0178 - val_loss: 0.0393
